In [14]:
#coding=utf-8
import numpy as np
import pandas as pd
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn import preprocessing  
min_max_scaler = preprocessing.MinMaxScaler()  
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import train_test_split

In [15]:
data = pd.read_csv("../inputnew/contest_ext_crd_cd_lnd_ovd.csv")
print(data.shape)
print(data.columns)
data.head()

(199644, 3)
Index(['REPORT_ID', 'LAST_MONTHS', 'AMOUNT'], dtype='object')


,REPORT_ID,LAST_MONTHS,AMOUNT
0,24820,1.0,440.0
1,24820,1.0,95.0
2,54038,1.0,10.0
3,54038,1.0,671.0
4,88203,1.0,1321.0


In [16]:
data.LAST_MONTHS.value_counts()

1.0    147563
2.0     24415
7.0     10061
3.0      8678
4.0      4309
5.0      2691
6.0      1927
Name: LAST_MONTHS, dtype: int64

In [17]:
def groupDeal(group):
    # print(group)
    LAST_MONTHS_max = group.LAST_MONTHS.max()
    LAST_MONTHS_min = group.LAST_MONTHS.min()
    LAST_MONTHS_mean = round(group.LAST_MONTHS.mean())
    LAST_MONTHS_zongshu = group.LAST_MONTHS.value_counts().iloc[0]
    LAST_MONTHS_num = group.LAST_MONTHS.shape[0]

    AMOUNT_max = group.AMOUNT.max()
    AMOUNT_min = group.AMOUNT.min()
    AMOUNT_mean = group.AMOUNT.mean()
    AMOUNT_zongshu = group.AMOUNT.value_counts().iloc[0]
    AMOUNT_num = group.AMOUNT.shape[0]


    indexs = {
        "REPORT_ID":int(group.REPORT_ID.iloc[0]),
        
        "LAST_MONTHS_max": LAST_MONTHS_max,
        "LAST_MONTHS_min": LAST_MONTHS_min,
        "LAST_MONTHS_mean": LAST_MONTHS_mean,
        "LAST_MONTHS_zongshu": LAST_MONTHS_zongshu,
        "LAST_MONTHS_num": LAST_MONTHS_num,

        "AMOUNT_max": AMOUNT_max,
        "AMOUNT_min": AMOUNT_min,
        "AMOUNT_mean": AMOUNT_mean,
        "AMOUNTt_zongshu": AMOUNT_zongshu,
        "AMOUNT_num": AMOUNT_num,
    }
    return pd.Series(data=[indexs[c] for c in indexs], index=[c for c in indexs])

In [19]:
# tmp1 = data.iloc[0:30,:].groupby(by=['REPORT_ID']).apply(groupDeal)
tmp1 = data.groupby(by=['REPORT_ID']).apply(groupDeal)
# tmp1
tmp1.to_csv("../add/willnorm/contest_ext_crd_cd_lnd_ovdRaw.csv",index = False)

In [20]:
data = pd.read_csv("../add/willnorm/contest_ext_crd_cd_lnd_ovdRaw.csv")
print(data.shape)
print(data.columns)
data.head()

(16429, 11)
Index(['REPORT_ID', 'LAST_MONTHS_max', 'LAST_MONTHS_min', 'LAST_MONTHS_mean',
       'LAST_MONTHS_zongshu', 'LAST_MONTHS_num', 'AMOUNT_max', 'AMOUNT_min',
       'AMOUNT_mean', 'AMOUNTt_zongshu', 'AMOUNT_num'],
      dtype='object')


,REPORT_ID,LAST_MONTHS_max,LAST_MONTHS_min,LAST_MONTHS_mean,LAST_MONTHS_zongshu,LAST_MONTHS_num,AMOUNT_max,AMOUNT_min,AMOUNT_mean,AMOUNTt_zongshu,AMOUNT_num
0,24820.0,1.0,1.0,1.0,2.0,2.0,440.0,95.0,267.500000,1.0,2.0
1,54038.0,1.0,1.0,1.0,2.0,2.0,671.0,10.0,340.500000,1.0,2.0
2,88203.0,2.0,1.0,1.0,9.0,10.0,1321.0,16.0,473.900000,1.0,10.0
3,125045.0,1.0,1.0,1.0,6.0,6.0,2526.0,39.0,1078.666667,2.0,6.0
4,128445.0,1.0,1.0,1.0,2.0,2.0,957.0,141.0,549.000000,1.0,2.0


In [21]:
label = data.iloc[:,1:].columns
# print(label)
number = data.iloc[:,1:]
kk = min_max_scaler.fit_transform(number)  
numbernew = pd.DataFrame(kk,columns = label)
numbernew.head()
fulldata = pd.concat([data.REPORT_ID,numbernew],axis = 1)
fulldata.head()

,REPORT_ID,LAST_MONTHS_max,LAST_MONTHS_min,LAST_MONTHS_mean,LAST_MONTHS_zongshu,LAST_MONTHS_num,AMOUNT_max,AMOUNT_min,AMOUNT_mean,AMOUNTt_zongshu,AMOUNT_num
0,24820.0,0.000000,0.0,0.0,0.008929,0.000000,0.001088,0.000960,0.001403,0.000000,0.000000
1,54038.0,0.000000,0.0,0.0,0.008929,0.000000,0.001660,0.000101,0.001787,0.000000,0.000000
2,88203.0,0.166667,0.0,0.0,0.071429,0.049383,0.003270,0.000162,0.002490,0.000000,0.049383
3,125045.0,0.000000,0.0,0.0,0.044643,0.024691,0.006255,0.000394,0.005674,0.027027,0.024691
4,128445.0,0.000000,0.0,0.0,0.008929,0.000000,0.002368,0.001424,0.002885,0.000000,0.000000


In [22]:
#删掉一些维度
train = pd.read_csv("../input/contest_basic_train.tsv",sep='\t')
mm = pd.concat([train.REPORT_ID,train.Y],axis = 1)
ceshidata = fulldata.merge(mm,how = 'inner',left_on = 'REPORT_ID',right_on = 'REPORT_ID') 
print(ceshidata.head())
print(ceshidata.shape)

  REPORT_ID  LAST_MONTHS_max  LAST_MONTHS_min  LAST_MONTHS_mean  \
0     24820         0.000000              0.0               0.0   
1     54038         0.000000              0.0               0.0   
2     88203         0.166667              0.0               0.0   
3    125045         0.000000              0.0               0.0   
4    128445         0.000000              0.0               0.0   

   LAST_MONTHS_zongshu  LAST_MONTHS_num  AMOUNT_max  AMOUNT_min  AMOUNT_mean  \
0             0.008929         0.000000    0.001088    0.000960     0.001403   
1             0.008929         0.000000    0.001660    0.000101     0.001787   
2             0.071429         0.049383    0.003270    0.000162     0.002490   
3             0.044643         0.024691    0.006255    0.000394     0.005674   
4             0.008929         0.000000    0.002368    0.001424     0.002885   

   AMOUNTt_zongshu  AMOUNT_num  Y  
0         0.000000    0.000000  1  
1         0.000000    0.000000  0  
2       

In [23]:
ceshidata.Y.value_counts()

0    12112
1      648
Name: Y, dtype: int64

In [8]:
# from imblearn.over_sampling import SMOTE, ADASYN
# X_resampled, y_resampled = SMOTE(kind='borderline1').fit_sample(ceshidata.iloc[:,1:-1], ceshidata.iloc[:,-1])

In [24]:

# print(X_resampled[0:10,:])
# print(X_resampled.shape)
# print(y_resampled[0:10])
# print(y_resampled.shape)
# pd.Series(y_resampled).value_counts()
# x_train,x_valid,y_train, y_valid= train_test_split(X_resampled,y_resampled,test_size = 0.3)
x_train,x_valid,y_train, y_valid= train_test_split(ceshidata.iloc[:,1:-1],ceshidata.iloc[:,-1],test_size = 0.3)
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(8932, 10)
(8932,)
(3828, 10)
(3828,)


In [32]:
pd.Series(y_train).value_counts()

0    6454
1     339
Name: Y, dtype: int64

In [25]:
# x_train = ceshidata.iloc[0:18000,1:-1]
# y_train = ceshidata.iloc[0:18000,-1]
# x_valid = ceshidata.iloc[18000:,1:-1]
# y_valid = ceshidata.iloc[18000:,-1]

skf = StratifiedKFold(n_splits=5)
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=3,
        scale_pos_weight=1,
        StratifiedKFold = True,
        seed=27)
xgb_param = xgb1.get_xgb_params()
#构建稀疏矩阵，运行更快
xgtrain = xgb.DMatrix(x_train, label = y_train)
# xgtest =xgb.DMatrix(dtest[predictors].values)
mm = xgb.cv(xgb_param, xgtrain, num_boost_round=1000, nfold=3,
    metrics='auc', early_stopping_rounds=50)
mm

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.562427,0.010375,0.564816,0.005106
1,0.562054,0.010627,0.566606,0.006645
2,0.560218,0.010842,0.578459,0.016510
3,0.559965,0.011100,0.578892,0.016542
4,0.560431,0.011150,0.581789,0.015061
5,0.569319,0.024718,0.615902,0.016550
6,0.567852,0.025820,0.632895,0.029090
7,0.566790,0.025594,0.639196,0.025940
8,0.571427,0.023118,0.649040,0.026418
9,0.570648,0.022742,0.668313,0.023454


In [26]:

print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(8932, 10)
(8932,)
(3828, 10)
(3828,)


In [28]:
feature = 0
for i in range(5):
    xgb2 = XGBClassifier(
            learning_rate =0.1,
            n_estimators=mm.shape[0],
            max_depth=5,
            min_child_weight=1,
            gamma=0,
            subsample=0.8,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,   #15:1
            StratifiedKFold = True,
            seed=27)
    xgb2.fit(x_train,y_train)
    feature += xgb2.feature_importances_
    print(i)
#     pred = xgb2.predict_proba(x_valid)[:,1]
#     print(pred)
#     auc = roc_auc_score(y_valid, pred)  
#     auc
feature = feature/5

0
1
2
3
4


In [29]:
feature

array([0.06857143, 0.01714286, 0.10285714, 0.02857143, 0.07428572,
       0.33142856, 0.16      , 0.16571428, 0.04      , 0.01142857],
      dtype=float32)

In [30]:
print(x_train.columns)
print(feature[feature>0.000008])
x_train.columns[feature>0.0000008].shape

Index(['LAST_MONTHS_max', 'LAST_MONTHS_min', 'LAST_MONTHS_mean',
       'LAST_MONTHS_zongshu', 'LAST_MONTHS_num', 'AMOUNT_max', 'AMOUNT_min',
       'AMOUNT_mean', 'AMOUNTt_zongshu', 'AMOUNT_num'],
      dtype='object')
[0.06857143 0.01714286 0.10285714 0.02857143 0.07428572 0.33142856
 0.16       0.16571428 0.04       0.01142857]


(10,)

In [31]:
fulldata.head()
fulldataok = []
fulldataok = fulldata[x_train.columns[feature>0.0000008]]  #一定要小心是谁的ID
fulldataok = pd.concat([fulldata.REPORT_ID,fulldataok],axis = 1)
fulldataok.head()

,REPORT_ID,LAST_MONTHS_max,LAST_MONTHS_min,LAST_MONTHS_mean,LAST_MONTHS_zongshu,LAST_MONTHS_num,AMOUNT_max,AMOUNT_min,AMOUNT_mean,AMOUNTt_zongshu,AMOUNT_num
0,24820.0,0.000000,0.0,0.0,0.008929,0.000000,0.001088,0.000960,0.001403,0.000000,0.000000
1,54038.0,0.000000,0.0,0.0,0.008929,0.000000,0.001660,0.000101,0.001787,0.000000,0.000000
2,88203.0,0.166667,0.0,0.0,0.071429,0.049383,0.003270,0.000162,0.002490,0.000000,0.049383
3,125045.0,0.000000,0.0,0.0,0.044643,0.024691,0.006255,0.000394,0.005674,0.027027,0.024691
4,128445.0,0.000000,0.0,0.0,0.008929,0.000000,0.002368,0.001424,0.002885,0.000000,0.000000


In [32]:
fulldataok.to_csv("../add/contest_ext_crd_cd_lnd_ovdok.csv",index = False)

In [33]:
fulldataok.shape

(16429, 11)